
### Project 1- Face Recognition System


In [1]:
import cv2
import pickle
import numpy as np
import os

# Create the 'data' directory if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

video = cv2.VideoCapture(0)
facedetect = cv2.CascadeClassifier(r'C:\\Users\\Hp\\Downloads\\haarcascade_frontalface_default.xml')

faces_data = []
i = 0

name = input("Enter Your Name: ")

while True:
    ret, frame = video.read()
    if not ret:
        print("Failed to capture video")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facedetect.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        crop_img = frame[y:y+h, x:x+w]
        resized_img = cv2.resize(crop_img, (50, 50))

        if len(faces_data) < 100 and i % 10 == 0:
            faces_data.append(resized_img)

        i += 1
        cv2.putText(frame, str(len(faces_data)), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (50, 50, 255), 1)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (50, 50, 255), 1)

    cv2.imshow("Frame", frame)
    
    k = cv2.waitKey(1)
    if k == ord('q') or len(faces_data) >= 100:
        break

video.release()
cv2.destroyAllWindows()

# Convert to numpy array and check if data is valid
faces_data = np.asarray(faces_data)

if faces_data.size == 0:
    print("No face data collected. Exiting.")
    exit()

# Ensure there are exactly 100 samples for reshaping
if len(faces_data) < 100:
    print(f"Only {len(faces_data)} faces detected. Padding to 100.")
    faces_data = np.pad(faces_data, ((0, 100 - len(faces_data)), (0, 0), (0, 0), (0, 0)), 'constant')

faces_data = faces_data.reshape(100, -1)  # Reshape to (100, 50*50*3)

# Save the names
if 'names.pkl' not in os.listdir('data/'):
    names = [name] * 100
    with open('data/names.pkl', 'wb') as f:
        pickle.dump(names, f)
else:
    with open('data/names.pkl', 'rb') as f:
        names = pickle.load(f)
    names += [name] * (100 - len(names))  # Ensure we only add what is needed
    with open('data/names.pkl', 'wb') as f:
        pickle.dump(names, f)

# Save the faces data
if 'faces_data.pkl' not in os.listdir('data/'):
    with open('data/faces_data.pkl', 'wb') as f:
        pickle.dump(faces_data, f)
else:
    with open('data/faces_data.pkl', 'rb') as f:
        faces = pickle.load(f)
    
    # Append only if the loaded faces array is valid
    if faces.size == 0:
        faces = faces_data
    else:
        faces = np.append(faces, faces_data, axis=0)

    with open('data/faces_data.pkl', 'wb') as f:
        pickle.dump(faces, f)

print("Data saved successfully!")

Enter Your Name: SHIVANI
Data saved successfully!


In [3]:
from sklearn.neighbors import KNeighborsClassifier
import cv2
import pickle
import numpy as np
import os
import csv
import time
from datetime import datetime
import win32com.client
from win32com.client import Dispatch

def speak(str1):
    speak = Dispatch("SAPI.SpVoice")
    speak.Speak(str1)

# Initialize video capture
video = cv2.VideoCapture(0)
facedetect = cv2.CascadeClassifier(r'C:\\Users\\Hp\\Downloads\\haarcascade_frontalface_default.xml')

# Load labels and face data
with open('data/names.pkl', 'rb') as w:
    LABELS = pickle.load(w)
with open('data/faces_data.pkl', 'rb') as f:
    FACES = pickle.load(f)

print('Shape of Faces matrix --> ', FACES.shape)

# Initialize KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(FACES, LABELS)

# Prepare attendance CSV column names
COL_NAMES = ['NAME', 'TIME']

while True:
    ret, frame = video.read()
    if not ret:
        print("Failed to grab frame.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facedetect.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        crop_img = frame[y:y+h, x:x+w]
        resized_img = cv2.resize(crop_img, (50, 50)).flatten().reshape(1, -1)
        output = knn.predict(resized_img)

        ts = time.time()
        date = datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
        timestamp = datetime.fromtimestamp(ts).strftime("%H-%M-%S")
        
        # Draw rectangles around detected faces
        cv2.rectangle(frame, (x, y), (x+w, y+h), (50, 50, 255), 2)
        cv2.putText(frame, str(output[0]), (x, y-15), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)

        attendance = [str(output[0]), str(timestamp)]

    cv2.imshow("Frame", frame)
    k = cv2.waitKey(1)

    if k == ord('p'):
        speak("Attendance Taken..")
        time.sleep(5)

        # Ensure the Attendance directory exists
        if not os.path.exists("Attendance"):
            os.makedirs("Attendance")

        # Construct the filename for the saved image
        img_filename = f"Attendance/Attendance_{output[0]}_{date}_{timestamp}.png"
        print(f"Saving image to {img_filename}")

        # Save the current frame as an image
        if frame is not None and frame.size > 0:
            success = cv2.imwrite(img_filename, frame)
            if not success:
                print(f"Error saving image to {img_filename}")
            else:
                print("Image saved successfully.")
        else:
            print("Invalid frame, not saving.")

        # Write attendance to CSV
        csv_file_path = f"Attendance/Attendance_{date}.csv"
        exist = os.path.isfile(csv_file_path)
        
        with open(csv_file_path, "a", newline='') as csvfile:
            writer = csv.writer(csvfile)
            if not exist:
                writer.writerow(COL_NAMES)
            writer.writerow(attendance)

    if k == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


Shape of Faces matrix -->  (100, 7500)
Saving image to Attendance/Attendance_SHIVANI_29-11-2024_15-18-35.png
Image saved successfully.
Saving image to Attendance/Attendance_SHIVANI_29-11-2024_15-18-43.png
Image saved successfully.
Saving image to Attendance/Attendance_SHIVANI_29-11-2024_15-18-50.png
Image saved successfully.
Saving image to Attendance/Attendance_SHIVANI_29-11-2024_15-18-57.png
Image saved successfully.
Saving image to Attendance/Attendance_SHIVANI_29-11-2024_15-19-04.png
Image saved successfully.
Saving image to Attendance/Attendance_SHIVANI_29-11-2024_15-19-11.png
Image saved successfully.


In [6]:
import streamlit as st
import pandas as pd
import time 
from datetime import datetime

ts=time.time()
date=datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
timestamp=datetime.fromtimestamp(ts).strftime("%H:%M-%S")

from streamlit_autorefresh import st_autorefresh

count = st_autorefresh(interval=2000, limit=100, key="fizzbuzzcounter")

if count == 0:
    st.write("Count is zero")
elif count % 3 == 0 and count % 5 == 0:
    st.write("FizzBuzz")
elif count % 3 == 0:
    st.write("Fizz")
elif count % 5 == 0:
    st.write("Buzz")
else:
    st.write(f"Count: {count}")


df=pd.read_csv("Attendance/Attendance_" + date + ".csv")

st.dataframe(df.style.highlight_max(axis=0))

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [7]:
df

,NAME,TIME
0,SHIVANI,15-18-00
1,SHIVANI,15-18-08
2,SHIVANI,15-18-35
3,SHIVANI,15-18-43
4,SHIVANI,15-18-50
5,SHIVANI,15-18-57
6,SHIVANI,15-19-04
7,SHIVANI,15-19-11
